# Project: No-show Appointment Data Analysis

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
<li><a href="#operations">Operations</a></li>
</ul>

<a id='intro'></a>
## Introduction

The dataset collects information from 100k medical appointments in Brazil. A number of characteristics about the patient are included.
>
<li>'PatientID' identifies a specific patient.
<li>'AppointmentID' identifies a specific appointment.
<li>'Gender' indicates the gender of the patient.
<li>‘ScheduledDay’ tells us the date the patient set up their appointment.
<li>'AppointmentDay' tells us the date of the appointment.
<li>'Age' indicates the age of the patient.
<li>‘Neighborhood’ indicates the location of the hospital.
<li>‘Scholarship’ indicates whether or not the patient is enrolled in Brasilian welfare program Bolsa Família.
<li>'Hipertension' indicates whether or not the patient is suffering from hipertension.
<li>'Diabetes' indicates whether or not the patient is suffering from diabetes.
<li>'Alcoholism' indicates whether or not the patient is suffering from alcoholism.
<li>'Handcap' indicates whether or not the patient has a disability.
<li>'SMS_received' tells us if the patient received an SMS upfront about the upcoming appointment.
<li>'No-show' indicates whether or not the patient showed up for their appointment.

This analysis focuses on data of patients that showed up or didn't show up for their medical appointments.
The intention is to find differences between the two groups.

### Questions
>
<li>Do no-shows for appointments differ depending on the age of the patient? Hypothesis: Older patients are more responsible and have a lower no-show rate.
<li>Do no-shows for appointments differ if patients received an SMS? Hypothesis: Patients that received an SMS are reminded of their appointment and have a lower no-show rate.
<li>Do no-shows for appointments differ if patients have a scholarship? Hypothesis: Patients with a scholarship have an insurance and therefore, don't need to worry about the expenses of an appointment and have a lower no-show rate.
<li>Do no-shows for appointments differ if the appointment is scheduled way in advance? Hypothesis: Patients that schedule their appointments way in advance, forget about them and have a higher no-show rate.

<a id='wrangling'></a>
## Data Wrangling

### General Properties

In [1]:
# import of packages 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [2]:
# load data 
df = pd.read_csv('noshowappointments-kagglev2-may-2016.csv')

In [3]:
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [5]:
df.dtypes

PatientId         float64
AppointmentID       int64
Gender             object
ScheduledDay       object
AppointmentDay     object
Age                 int64
Neighbourhood      object
Scholarship         int64
Hipertension        int64
Diabetes            int64
Alcoholism          int64
Handcap             int64
SMS_received        int64
No-show            object
dtype: object

In [6]:
df.select_dtypes(include=int) #int, float, bool
#df.select_dtypes(exclude=object)

,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
0,5642903,62,0,1,0,0,0,0
1,5642503,56,0,0,0,0,0,0
2,5642549,62,0,0,0,0,0,0
3,5642828,8,0,0,0,0,0,0
4,5642494,56,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...
110522,5651768,56,0,0,0,0,0,1
110523,5650093,51,0,0,0,0,0,1
110524,5630692,21,0,0,0,0,0,1
110525,5630323,38,0,0,0,0,0,1


In [7]:
df.describe()

,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
count,1.105270e+05,1.105270e+05,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,1.474963e+14,5.675305e+06,37.088874,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026
std,2.560949e+14,7.129575e+04,23.110205,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873
min,3.921784e+04,5.030230e+06,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172614e+12,5.640286e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173184e+13,5.680573e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.439172e+13,5.725524e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999816e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


In [8]:
df.duplicated().sum()

0

In [9]:
column_list = df.columns
column_list = column_list.tolist()

In [10]:
df.rename(columns={'PatientId':'patient_id', 'AppointmentID':'appointment_id', 'Gender':'gender', 
                   'ScheduledDay':'scheduled_day', 'AppointmentDay':'appointment_day', 'Age':'age', 
                   'Neighbourhood':'neighbourhood', 'Scholarship':'scholarship', 'Hipertension':'hipertension',
                   'Diabetes':'diabetes', 'Alcoholism':'alcoholism', 'Handcap':'handicap', 'SMS_received':'messaged',
                   'No-show':'no_show'}, inplace=True)

In [11]:
df.duplicated(subset='patient_id').sum()

48228

In [12]:
for i in range(0, len(df.columns)):
    print(df.columns[i])
    print(df.duplicated(subset=df.columns[i]).sum())

patient_id
48228
appointment_id
0
gender
110525
scheduled_day
6978
appointment_day
110500
age
110423
neighbourhood
110446
scholarship
110525
hipertension
110525
diabetes
110525
alcoholism
110525
handicap
110522
messaged
110525
no_show
110525


In [13]:
for i in range(0, len(df.columns)):
    print(df[df.columns[i]].value_counts())

8.221459e+14    88
9.963767e+10    84
2.688613e+13    70
3.353478e+13    65
7.579746e+13    62
                ..
1.779297e+13     1
9.985120e+11     1
3.256827e+13     1
9.232297e+13     1
5.133834e+14     1
Name: patient_id, Length: 62299, dtype: int64
5771266    1
5680512    1
5602682    1
5598584    1
5584243    1
          ..
5686642    1
5692785    1
5647727    1
5645678    1
5769215    1
Name: appointment_id, Length: 110527, dtype: int64
F    71840
M    38687
Name: gender, dtype: int64
2016-05-06T07:09:54Z    24
2016-05-06T07:09:53Z    23
2016-04-25T17:17:46Z    22
2016-04-25T17:18:27Z    22
2016-04-25T17:17:23Z    19
                        ..
2016-05-16T07:59:18Z     1
2016-05-18T16:47:16Z     1
2016-06-03T06:34:11Z     1
2016-04-14T12:38:02Z     1
2016-05-11T15:54:54Z     1
Name: scheduled_day, Length: 103549, dtype: int64
2016-06-06T00:00:00Z    4692
2016-05-16T00:00:00Z    4613
2016-05-09T00:00:00Z    4520
2016-05-30T00:00:00Z    4514
2016-06-08T00:00:00Z    4479
2016-05-11

In [14]:
df.scheduled_day.min()

'2015-11-10T07:13:56Z'

In [15]:
df.scheduled_day.max()

'2016-06-08T20:07:23Z'

In [16]:
df.appointment_day.min()

'2016-04-29T00:00:00Z'

In [17]:
df.appointment_day.max()

'2016-06-08T00:00:00Z'

In [18]:
df.age.describe()

count    110527.000000
mean         37.088874
std          23.110205
min          -1.000000
25%          18.000000
50%          37.000000
75%          55.000000
max         115.000000
Name: age, dtype: float64

In [19]:
df_numpy = df.to_numpy
df_numpy

<bound method DataFrame.to_numpy of           patient_id  appointment_id gender         scheduled_day  \
0       2.987250e+13         5642903      F  2016-04-29T18:38:08Z   
1       5.589978e+14         5642503      M  2016-04-29T16:08:27Z   
2       4.262962e+12         5642549      F  2016-04-29T16:19:04Z   
3       8.679512e+11         5642828      F  2016-04-29T17:29:31Z   
4       8.841186e+12         5642494      F  2016-04-29T16:07:23Z   
...              ...             ...    ...                   ...   
110522  2.572134e+12         5651768      F  2016-05-03T09:15:35Z   
110523  3.596266e+12         5650093      F  2016-05-03T07:27:33Z   
110524  1.557663e+13         5630692      F  2016-04-27T16:03:52Z   
110525  9.213493e+13         5630323      F  2016-04-27T15:09:23Z   
110526  3.775115e+14         5629448      F  2016-04-27T13:30:56Z   

             appointment_day  age      neighbourhood  scholarship  \
0       2016-04-29T00:00:00Z   62    JARDIM DA PENHA            0 

In [20]:
df.values[0]

array([29872499824296.0, 5642903, 'F', '2016-04-29T18:38:08Z',
       '2016-04-29T00:00:00Z', 62, 'JARDIM DA PENHA', 0, 1, 0, 0, 0, 0,
       'No'], dtype=object)

In [21]:
df.axes

[RangeIndex(start=0, stop=110527, step=1),
 Index(['patient_id', 'appointment_id', 'gender', 'scheduled_day',
        'appointment_day', 'age', 'neighbourhood', 'scholarship',
        'hipertension', 'diabetes', 'alcoholism', 'handicap', 'messaged',
        'no_show'],
       dtype='object')]

In [22]:
df.shape

(110527, 14)

In [23]:
df.memory_usage()

Index                 128
patient_id         884216
appointment_id     884216
gender             884216
scheduled_day      884216
appointment_day    884216
age                884216
neighbourhood      884216
scholarship        884216
hipertension       884216
diabetes           884216
alcoholism         884216
handicap           884216
messaged           884216
no_show            884216
dtype: int64

In [24]:
df.keys()

Index(['patient_id', 'appointment_id', 'gender', 'scheduled_day',
       'appointment_day', 'age', 'neighbourhood', 'scholarship',
       'hipertension', 'diabetes', 'alcoholism', 'handicap', 'messaged',
       'no_show'],
      dtype='object')

### Data Cleaning 

<li> Adjust column names
<li> Adjust data types
<li> Find na values
<li> Adjust unrealistic values for age
<li> Adjust values for neighbourhood (title)
<li> Adjust values for neighbourhood (Santa Lucia and Santa Luzia)
<li> Adjust no_show values (0,1)
<li> Adjust gender values (0,1)

In [25]:
# adjust column names
df.columns
df.rename(columns={'PatientId':'patient_id', 'AppointmentID':'appointment_id', 'Gender':'gender', 
                   'ScheduledDay':'scheduled_day', 'AppointmentDay':'appointment_day', 'Age':'age', 
                   'Neighbourhood':'neighbourhood', 'Scholarship':'scholarship', 'Hipertension':'hipertension',
                   'Diabetes':'diabetes', 'Alcoholism':'alcoholism', 'Handcap':'handicap', 'SMS_received':'messaged',
                   'No-show':'no_show'}, inplace=True)
df.head()

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handicap,messaged,no_show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [26]:
# adjust data types
df.scheduled_day = pd.to_datetime(df.scheduled_day) 
df.appointment_day = pd.to_datetime(df.appointment_day) 

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   patient_id       110527 non-null  float64            
 1   appointment_id   110527 non-null  int64              
 2   gender           110527 non-null  object             
 3   scheduled_day    110527 non-null  datetime64[ns, UTC]
 4   appointment_day  110527 non-null  datetime64[ns, UTC]
 5   age              110527 non-null  int64              
 6   neighbourhood    110527 non-null  object             
 7   scholarship      110527 non-null  int64              
 8   hipertension     110527 non-null  int64              
 9   diabetes         110527 non-null  int64              
 10  alcoholism       110527 non-null  int64              
 11  handicap         110527 non-null  int64              
 12  messaged         110527 non-null  int64              
 13 

In [27]:
# adjust string values with integer values (no_show)
df.no_show = df.no_show.replace(['Yes', 'No'], [0, 1])

df.no_show.value_counts()

1    88208
0    22319
Name: no_show, dtype: int64

In [28]:
df.gender = df.gender.replace(['F', 'M'], [0, 1])

df.gender.value_counts()

0    71840
1    38687
Name: gender, dtype: int64

In [29]:
# adjust unrealistic values for age 

age = df.age.value_counts()
age = pd.DataFrame(age).reset_index().rename(columns={'index':'age', 'age':'count'}).sort_values(by=('age'))
age

# values below 0 and above 100 seem unrealistic

df = df.drop(df[df.age > 100].index)
df = df.drop(df[df.age < 0].index)

df.age.value_counts()

0      3539
1      2273
52     1746
49     1652
53     1651
       ... 
96       17
97       11
98        6
100       4
99        1
Name: age, Length: 101, dtype: int64

In [30]:
# adjust neighbourhood

df.neighbourhood = df.neighbourhood.str.lower()
df.neighbourhood = df.neighbourhood.str.title()

df.head()

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handicap,messaged,no_show
0,2.987250e+13,5642903,0,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,62,Jardim Da Penha,0,1,0,0,0,0,1
1,5.589978e+14,5642503,1,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,56,Jardim Da Penha,0,0,0,0,0,0,1
2,4.262962e+12,5642549,0,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,62,Mata Da Praia,0,0,0,0,0,0,1
3,8.679512e+11,5642828,0,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8,Pontal De Camburi,0,0,0,0,0,0,1
4,8.841186e+12,5642494,0,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,56,Jardim Da Penha,0,1,1,0,0,0,1


<a id='eda'></a>
## Exploratory Data Analysis

### Question 1

In [31]:
# replace values where condition is false

df.gender = df.gender.where(df.gender==1) # replace with NaN
df.gender = df.gender.where(df.gender==1, 10) # replace with 10

In [32]:
# replace values where condition is true

df.age = df.age.mask(df.age == 62) # replace with NaN
df.age = df.age.mask(df.age == 56, 10) # replace with 10

In [33]:
df.query('gender > hipertension')
df[df.gender > df.hipertension]
df[df['gender'] > df['hipertension']]

# similar expressions

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handicap,messaged,no_show
0,2.987250e+13,5642903,10.0,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,NaN,Jardim Da Penha,0,1,0,0,0,0,1
1,5.589978e+14,5642503,1.0,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,10.0,Jardim Da Penha,0,0,0,0,0,0,1
2,4.262962e+12,5642549,10.0,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,NaN,Mata Da Praia,0,0,0,0,0,0,1
3,8.679512e+11,5642828,10.0,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8.0,Pontal De Camburi,0,0,0,0,0,0,1
4,8.841186e+12,5642494,10.0,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,10.0,Jardim Da Penha,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,2.572134e+12,5651768,10.0,2016-05-03 09:15:35+00:00,2016-06-07 00:00:00+00:00,10.0,Maria Ortiz,0,0,0,0,0,1,1
110523,3.596266e+12,5650093,10.0,2016-05-03 07:27:33+00:00,2016-06-07 00:00:00+00:00,51.0,Maria Ortiz,0,0,0,0,0,1,1
110524,1.557663e+13,5630692,10.0,2016-04-27 16:03:52+00:00,2016-06-07 00:00:00+00:00,21.0,Maria Ortiz,0,0,0,0,0,1,1
110525,9.213493e+13,5630323,10.0,2016-04-27 15:09:23+00:00,2016-06-07 00:00:00+00:00,38.0,Maria Ortiz,0,0,0,0,0,1,1


#### Comparison

DataFrame.eq
Compare DataFrames for equality elementwise.

DataFrame.ne
Compare DataFrames for inequality elementwise.

DataFrame.le
Compare DataFrames for less than inequality or equality elementwise.

DataFrame.lt
Compare DataFrames for strictly less than inequality elementwise.

DataFrame.ge
Compare DataFrames for greater than inequality or equality elementwise.

DataFrame.gt
Compare DataFrames for strictly greater than inequality elementwise.

In [34]:
df.eq(10)

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handicap,messaged,no_show
0,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False,False,False,False,False
4,False,False,True,False,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,False,False,True,False,False,True,False,False,False,False,False,False,False,False
110523,False,False,True,False,False,False,False,False,False,False,False,False,False,False
110524,False,False,True,False,False,False,False,False,False,False,False,False,False,False
110525,False,False,True,False,False,False,False,False,False,False,False,False,False,False


In [35]:
df.head()

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handicap,messaged,no_show
0,2.987250e+13,5642903,10.0,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,NaN,Jardim Da Penha,0,1,0,0,0,0,1
1,5.589978e+14,5642503,1.0,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,10.0,Jardim Da Penha,0,0,0,0,0,0,1
2,4.262962e+12,5642549,10.0,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,NaN,Mata Da Praia,0,0,0,0,0,0,1
3,8.679512e+11,5642828,10.0,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8.0,Pontal De Camburi,0,0,0,0,0,0,1
4,8.841186e+12,5642494,10.0,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,10.0,Jardim Da Penha,0,1,1,0,0,0,1


In [36]:
df.head()

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handicap,messaged,no_show
0,2.987250e+13,5642903,10.0,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,NaN,Jardim Da Penha,0,1,0,0,0,0,1
1,5.589978e+14,5642503,1.0,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,10.0,Jardim Da Penha,0,0,0,0,0,0,1
2,4.262962e+12,5642549,10.0,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,NaN,Mata Da Praia,0,0,0,0,0,0,1
3,8.679512e+11,5642828,10.0,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8.0,Pontal De Camburi,0,0,0,0,0,0,1
4,8.841186e+12,5642494,10.0,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,10.0,Jardim Da Penha,0,1,1,0,0,0,1


In [37]:
# difference scheduled day and appoinment day

df['time_difference'] = df.appointment_day - df.scheduled_day

In [38]:
df.head()

,patient_id,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handicap,messaged,no_show,time_difference
0,2.987250e+13,5642903,10.0,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,NaN,Jardim Da Penha,0,1,0,0,0,0,1,-1 days +05:21:52
1,5.589978e+14,5642503,1.0,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,10.0,Jardim Da Penha,0,0,0,0,0,0,1,-1 days +07:51:33
2,4.262962e+12,5642549,10.0,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,NaN,Mata Da Praia,0,0,0,0,0,0,1,-1 days +07:40:56
3,8.679512e+11,5642828,10.0,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8.0,Pontal De Camburi,0,0,0,0,0,0,1,-1 days +06:30:29
4,8.841186e+12,5642494,10.0,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,10.0,Jardim Da Penha,0,1,1,0,0,0,1,-1 days +07:52:37


In [39]:
# plot ages
# plot patient ids

In [40]:
# difference in missed and not missed appointments and hypothesis testing

In [41]:
# more values for same patient id / different action?

In [42]:
# look at frequent patients

In [43]:
len_patient_id = len(df.patient_id.value_counts())
len_patient_id

62294

In [44]:
frequent_patients = []

for i in range(0, len_patient_id):
    if df.patient_id.value_counts().iloc[i] > 10:
        frequent_patients.append(df.patient_id.iloc[i])

In [45]:
len(frequent_patients)

248

In [46]:
neighbourhood = pd.DataFrame(df.neighbourhood.value_counts()).reset_index().rename(columns={'index':'neighbourhood', 'neighbourhood':'count'})
neighbourhood.neighbourhood = neighbourhood.neighbourhood.str.lower()
neighbourhood.neighbourhood = neighbourhood.neighbourhood.str.title()
neighbourhood = neighbourhood.sort_values(by=['neighbourhood'])
neighbourhood

,neighbourhood,count
78,Aeroporto,8
17,Andorinhas,2258
66,Antônio Honório,271
65,Ariovaldo Favalessa,282
59,Barro Vermelho,423
...,...,...
21,São José,1976
14,São Pedro,2448
6,Tabuazeiro,3132
71,Universitário,152


<a id='conclusions'></a>
## Conclusions

### Limitations



### Question 1:

Hypothesis: 



<a id='operations'></a>

## Operations

In [47]:
# setting values
# dfs.loc[['viper', 'sidewinder'], ['shield']] = 50
# dfs.loc['cobra'] = 10 # for all columns
# dfs.loc[:, 'max_speed'] = 30 # for complete column
# dfs.loc[df['shield'] > 35] = 0 # for column with condition

In [48]:
# insert columns in dataframe

dfe = pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]})
dfe

,col1,col2
0,1,3
1,2,4


In [49]:
dfe.insert(1, "newcol", [99, 99])
dfe

,col1,newcol,col2
0,1,99,3
1,2,99,4


In [50]:
dfe.insert(0, "col1", [100, 100], allow_duplicates=True)
dfe

,col1,col1,newcol,col2
0,100,1,99,3
1,100,2,99,4


In [51]:
dfe.insert(0, "col0", pd.Series([5, 6], index=[1, 2]))
dfe

,col0,col1,col1,newcol,col2
0,NaN,100,1,99,3
1,5.0,100,2,99,4


In [52]:
# use items (label, content)
dfi = pd.DataFrame({'species': ['bear', 'bear', 'marsupial'],
                  'population': [1864, 22000, 80000]},
                  index=['panda', 'polar', 'koala'])
dfi

,species,population
panda,bear,1864
polar,bear,22000
koala,marsupial,80000


In [53]:
for label, content in dfi.items():
    print(f'label: {label}')
    print(f'content: {content}', sep='\n')

label: species
content: panda         bear
polar         bear
koala    marsupial
Name: species, dtype: object
label: population
content: panda     1864
polar    22000
koala    80000
Name: population, dtype: int64


In [54]:
# pop item (delete from dataframe and keep)
df.pop('patient_id')

0         2.987250e+13
1         5.589978e+14
2         4.262962e+12
3         8.679512e+11
4         8.841186e+12
              ...     
110522    2.572134e+12
110523    3.596266e+12
110524    1.557663e+13
110525    9.213493e+13
110526    3.775115e+14
Name: patient_id, Length: 110519, dtype: float64

In [55]:
df.head()

,appointment_id,gender,scheduled_day,appointment_day,age,neighbourhood,scholarship,hipertension,diabetes,alcoholism,handicap,messaged,no_show,time_difference
0,5642903,10.0,2016-04-29 18:38:08+00:00,2016-04-29 00:00:00+00:00,NaN,Jardim Da Penha,0,1,0,0,0,0,1,-1 days +05:21:52
1,5642503,1.0,2016-04-29 16:08:27+00:00,2016-04-29 00:00:00+00:00,10.0,Jardim Da Penha,0,0,0,0,0,0,1,-1 days +07:51:33
2,5642549,10.0,2016-04-29 16:19:04+00:00,2016-04-29 00:00:00+00:00,NaN,Mata Da Praia,0,0,0,0,0,0,1,-1 days +07:40:56
3,5642828,10.0,2016-04-29 17:29:31+00:00,2016-04-29 00:00:00+00:00,8.0,Pontal De Camburi,0,0,0,0,0,0,1,-1 days +06:30:29
4,5642494,10.0,2016-04-29 16:07:23+00:00,2016-04-29 00:00:00+00:00,10.0,Jardim Da Penha,0,1,1,0,0,0,1,-1 days +07:52:37


In [56]:
# isin
dfis = pd.DataFrame({'num_legs': [2, 4], 'num_wings': [2, 0]},
                  index=['falcon', 'dog'])
dfis

,num_legs,num_wings
falcon,2,2
dog,4,0


In [57]:
dfis.isin([0, 2])

,num_legs,num_wings
falcon,True,True
dog,False,True


In [58]:
dfis.isin({'num_wings': [0, 3]})
# When values is a dict, we can pass values to check for each column separately

,num_legs,num_wings
falcon,False,False
dog,False,True


In [59]:
other = pd.DataFrame({'num_legs': [8, 2], 'num_wings': [0, 2]},
                     index=['spider', 'falcon'])

dfis.isin(other)

# When values is a Series or DataFrame the index and column must match. Note that ‘falcon’ does not match based on 
# the number of legs in df2.

,num_legs,num_wings
falcon,True,True
dog,False,False


In [60]:
# Perform column-wise combine with another DataFrame

df1 = pd.DataFrame({'A': [0, 0], 'B': [4, 4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
take_smaller = lambda s1, s2: s1 if s1.sum() < s2.sum() else s2
df1.combine(df2, take_smaller)


,A,B
0,0,3
1,0,3


In [61]:
df1 = pd.DataFrame({'A': [5, 0], 'B': [2, 4]})
df2 = pd.DataFrame({'A': [1, 1], 'B': [3, 3]})
df1.combine(df2, np.minimum)

,A,B
0,1,2
1,0,3


In [63]:
df1.apply(np.sqrt)

,A,B
0,2.236068,1.414214
1,0.000000,2.000000


In [65]:
df1.agg(['sum'])

,A,B
sum,5,6


In [67]:
df1.agg('sum', axis='columns')

0    7
1    4
dtype: int64

In [68]:
df1.transform([np.sqrt])

,A,B
,sqrt,sqrt
0,2.236068,1.414214
1,0.000000,2.000000


### Current status

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.combine.html